In [1]:
#FULL CODE:
#Importing libraries
import sys
import os
import re
import time

import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import spacy

from spacy import displacy
from spacy.tokens import DocBin
from spacy.tokens import Span

tqdm.pandas()
#using pretrained gensim model for creating vocabulary
import gensim.downloader as api
import gensim


In [2]:
import gensim
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
import numpy as np
import copy

In [3]:
#Loading different embeddings function by choice #files located locally
def load_embed(embed):
    if embed == "fasttext":
        return  (KeyedVectors.load('saved_ft_embed',mmap='r'))
    elif embed == "glove":
        return  (KeyedVectors.load('saved_glove_embed',mmap='r'))
    elif embed == "conceptnet":
        return (KeyedVectors.load(r"C:\Users\DELL\Desktop\NLP files\Ner model files\conceptnet_embeddings\saved_numb_batch_embed",mmap='r'))
    else:   
        return None

In [4]:
# cosine similarity function
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

In [5]:
def preprocessCorpus(corpus_file):
    import numpy as np
    import pandas as pd
    
    if corpus_file.endswith('csv'):
        corpus_df=pd.read_csv(corpus_file)
    elif corpus_file.endswith('xlsx'):
        corpus_df=pd.read_excel(corpus_file)
    
    return corpus_df

def preprocessVocab(vocab_file): 
    import numpy as np
    import pandas as pd
    if vocab_file.endswith('csv'):
        vocab_df=pd.read_csv(vocab_file)
    elif vocab_file.endswith('xlsx'):
        vocab_df=pd.read_excel(vocab_file)
    lst = vocab_df.columns.to_list()
    keyword_label_dict = vocab_df.set_index(lst[2]).to_dict()[lst[0]]   
        
    return keyword_label_dict

### Creating the nounchunk-embedding file

In [6]:
def processStr(string):
    string=" ".join(string.lower().replace('-',' ').split())
    return string

In [7]:
def nounchunk_embed(corpus_df):
    nlp = spacy.load("en_core_web_lg")
    all_stopwords = nlp.Defaults.stop_words
    embed_model = load_embed('conceptnet')
    noun_chunk_dict={}
    for text in tqdm(corpus_df.iloc[:, 0]):
        doc = nlp(text)
        for chunk in doc.noun_chunks:
            
            if str(chunk.root.text) in embed_model.key_to_index and str(chunk) not in all_stopwords:
                noun_chunk_dict[processStr(str(chunk))]=embed_model[str(chunk.root.text)]
    
    return noun_chunk_dict  # format ==> {'noun_chunk_1' : embedding ,'noun_chunk_2' : embedding , ----}

In [8]:
corp_file=r"C:\Users\DELL\Downloads\CE_GF_corpus_small_40 (1).csv"
co_df=preprocessCorpus(corp_file)
ncem_Dict=nounchunk_embed(co_df)
co_df.head()

  0%|          | 0/40 [00:00<?, ?it/s]

,title+body
0,Shelf life extension?Woodside and its joint ve...
1,Deals - Buying into a green futureSo it follow...
2,Nats signal new front in climate change war“Th...
3,Island nation's landmark legal challenge on cl...
4,Biden terms US Supreme Court's ruling in clima...


In [9]:
print(len(ncem_Dict))

12219


In [10]:
# # output --- nounchunks and embedding
lst=list(ncem_Dict.keys())
print(lst[:50])

['shelf life', 'its joint venture partners', 'the life', 'third party gas', 'the north west shelf fields', 'ministerial approval', 'additional measures', 'the mitigation', 'greenhouse gas emissions', 'the extension proposal', '385 million tonnes', 'its life', 'this assessment', 'a reduction', 'the proponent', 'the total quantity', 'reservoir carbon dioxide emissions', 'the atmosphere', 'the issue date', 'any ministerial approval', 'the recommended conditions', 'net scope 1 greenhouse gas emissions', 'the 50 year life', 'the project', 'significant opposition', 'legal challenges', 'climate activists', 'its proposed $16 billion scarborough gas project', 'its scope', '1 and 2 greenhouse gas emissions', 'an aspiration', 'targets', 'a 15 per cent reduction', '30 per cent', 'woodside’s executive vice president', 'australian operations', 'the conditions', 'a time', 'heightened concern', 'energy security', 'an important role', 'natural gas', 'local and international customers', 'energy', 'their

### Creating the keyword-embedding file

In [11]:
keyword_df=pd.read_excel(r"C:\Users\DELL\Downloads\ESG_keywords_29_7_22_v1_groupby_1_rem_different_forms.xlsx")
keyword_df.head()

,Label,NER(manually Produced),Embedding Extended Keywords(30),Similarity score,Doubtful
0,GREEN PERFORMANCE,percentage,%,0.523927,NaN
1,GREEN PERFORMANCE,percentage,absentee rate,0.997618,NaN
2,GREEN COMMITMENT,indicators,absorption indicator,0.726753,NaN
3,GREEN COMMITMENT,agree,accept,0.676722,NaN
4,GREEN PERFORMANCE,achieve,accomplish,0.815839,NaN


In [12]:
#######################################

def keywordembed(keyword_label_dict):
    embed_model = load_embed('conceptnet')
    keyword_embed_dict={}
    keyword_dict={}
    for keyword,label in keyword_label_dict.items():#{key : lab }
        keyword_embed_dict[label]=[] # ked= {"l1":[],"l2",[]}
        
    for keyword,label in tqdm(keyword_label_dict.items()):   
        _keyword = keyword.replace(' ','_')
        if _keyword in embed_model.key_to_index:
            keyword_embed_dict[label].append(embed_model[_keyword])
    return keyword_embed_dict      
        
    

In [13]:
vo_file=r"C:\Users\DELL\Downloads\ESG_keywords_29_7_22_v1_groupby_1_rem_different_forms.xlsx"
keyword_label_dict=preprocessVocab(vo_file)
keyem_dict=keywordembed(keyword_label_dict)
print(keyem_dict['PERSON ROLE'])
#print(sample_keyem_dict)

  0%|          | 0/848 [00:00<?, ?it/s]

[memmap([ 0.02710034,  0.07910098, -0.15140189, -0.04620057, -0.04670058,
         0.06120076, -0.09250116, -0.03860048, -0.06240078,  0.05000062,
        -0.06580082,  0.11590145, -0.17460218, -0.03160039,  0.03350042,
         0.0327004 ,  0.03310042, -0.07480093, -0.03660046,  0.07060088,
        -0.0721009 , -0.0404005 , -0.07020088, -0.02280028,  0.06170077,
         0.09950124, -0.0083001 ,  0.11170139,  0.0324004 , -0.06560082,
         0.05730072,  0.07970099, -0.04580057,  0.09410117,  0.12750159,
         0.01870023, -0.01000012,  0.07690096,  0.00910011,  0.12580156,
        -0.02220028,  0.01670021,  0.04640058,  0.00470006,  0.02490031,
        -0.00500006,  0.01440018,  0.02040025, -0.03980049,  0.05730072,
        -0.0563007 , -0.00100001,  0.00930012,  0.02670033,  0.00990012,
         0.01140014, -0.09410117, -0.06720083,  0.09400117,  0.05580069,
         0.05850073, -0.00280003,  0.01430018,  0.09460118, -0.07780097,
        -0.0240003 ,  0.03670046,  0.10360129, -0.

In [16]:
print(keyword_label_dict)

{'%': 'GREEN PERFORMANCE', 'absentee rate': 'GREEN PERFORMANCE', 'absorption indicator': 'GREEN COMMITMENT', 'accept': 'GREEN COMMITMENT', 'accomplish': 'GREEN PERFORMANCE', 'accord': 'GREEN PERFORMANCE', 'accountability': 'GREEN COMMITMENT', 'accounting': 'INVESTMENT', 'accrease': 'GREEN PERFORMANCE', 'acheive': 'GREEN PERFORMANCE', 'achieve': 'GREEN COMMITMENT', 'achievement': 'GREEN PERFORMANCE', 'achievement test': 'GREEN PERFORMANCE', 'acid of air': 'GREEN PARAMETER', 'acknowledge': 'GREEN PERFORMANCE', 'acquisition': 'INVESTMENT', 'acquittance': 'GREEN PERFORMANCE', 'acrid': 'GREEN PARAMETER', 'afp': 'PERSON ROLE', 'afterdischarge': 'GREEN PERFORMANCE', 'agent general': 'PERSON ROLE', 'agree': 'GREEN COMMITMENT', 'agreement': 'INVESTMENT', 'agrivoltaic': 'GREEN PARAMETER', 'AGW': 'GREEN PARAMETER', 'aim': 'GREEN COMMITMENT', 'aimed': 'GREEN PERFORMANCE', 'alarmists': 'GREEN PARAMETER', 'allegience': 'GREEN COMMITMENT', 'alliance': 'INVESTMENT', 'almost': 'GREEN COMMITMENT', 'alph

### Chunk comparision

In [ ]:
def chunk_comparision(noun_chunk_dict,keyword_embed_dict):
    similarity_dict={}
    for noun_chunk in tqdm(noun_chunk_dict): # {nc : em}
        k={}
        for label in keyword_embed_dict: # { l " []"}
            l=[]
            for kem in keyword_embed_dict[label]:
                l.append(get_cosine_similarity(noun_chunk_dict[noun_chunk],kem))
            k[label]=max(l)
        similarity_dict[noun_chunk]=max(k,key=k.get)
    return similarity_dict

In [ ]:
%%time
si_dict = chunk_comparision(nc_Dict,sample_keyem_dict)

In [ ]:
print(len(si_dict))

In [ ]:
final_op_dict=swapDict(si_dict)

In [ ]:
# import json
# path=r"C:\Users\DELL\Desktop\ESG_Keyword_5labels_nChunks_2_8_2022.json"
# with open(path, 'w') as f:
#     json.dump(output_dict, f)

In [ ]:
print(final_op_dict.keys())

In [ ]:
for value in final_op_dict.values():
    print(value[:20])

In [ ]:
def mainfunction(corpus_file,vocab_file):
    import numpy as np
    import pandas as pd
    # CREATING NOUN CHUNKS EMBEDDING FROM CORPUS
    # Preprocessing corpus_file
    corpus_df = preprocessCorpus(corpus_file)
    # Creating dictionary with embeddings
    noun_chunk_dict = nounchunk_embed(corpus_df)
    
    # CREATING KEYWORDS EMBEDDING FROM VOCAB
    vocab_df=preprocessVocab(vocab_file)
    lst = vocab_df.columns.to_list()
    keyword_label_dict = vocab_df.set_index(lst[0]).to_dict()[lst[1]]
    keyword_embed_dict = keywordembed(keyword_label_dict)                                    
                                        
    noun_chunk_label_dict = chunk_comparision(noun_chunk_dict,keyword_embed_dict)
    
    return noun_chunk_label_dict

In [ ]:
ip_corpus_file=r"C:\Users\DELL\Downloads\CE_GF_corpus_small_40 (1).csv"
ip_vocab_file=r"C:\Users\DELL\Desktop\Embedding_Extended_Keywords(30)_2_8_2022.csv"
final_dict=mainfunction(ip_corpus_file,ip_vocab_file)

In [ ]:
print(final_dict)

In [ ]:
for value in final_dict.values():
    print(value[:10])

In [ ]:
def swapDict(dictionary):    
    converted_dict = {} #swapping the
    for key, value in dictionary.items():
        if value in converted_dict:
            converted_dict[value].append(key)
        else:
            converted_dict[value]=[key]
    return converted_dict

In [15]:
ncd={"nc1":2 ,"nc2":0.5 ,"nc3":6 ,"nc4":0.5,"nc5":2.2}
ked={2:'l1', 0.2:'l1', 4:'l1', 3:'l1', 4:'l2', 2.2:'l2', 0.9:'l2', 3:'l3', 1:'l3', 2.3:'l3', 1.5:'l3', 2.8:'l3'}
sd={"l1":{},'l2':{},'l3':{}}
for nc,nce in ncd.items():
    for ke,kl in ked.items():
        ss=int(nce*ke)
        if ss > 2:
            sd[kl][ss]=nc#.append((nc,int(ss)))
print(sd)


{'l1': {4: 'nc5', 12: 'nc3'}, 'l2': {8: 'nc5', 4: 'nc5', 24: 'nc3', 13: 'nc3', 5: 'nc3'}, 'l3': {6: 'nc5', 4: 'nc1', 3: 'nc5', 5: 'nc5', 18: 'nc3', 13: 'nc3', 9: 'nc3', 16: 'nc3'}}


In [46]:
ncd={"nc1":2 ,"nc2":0.5 ,"nc3":6 ,"nc4":0.5,"nc5":2.2}
ked={"l1":[2, 0.2, 4.1, 3,4],"l2":[6, 2.2 , 0.9],"l3":[0.7,1,2.3,1.5,2.8]}
sd={} 
for nc in ncd:
    k={}
    for lab in ked:
        for kem in ked[lab]:
            ss=int(ncd[nc]*kem)
            if ss > 3:
                k[ss]=lab
    sd[nc]=k
print(sd)            

{'nc1': {4: 'l3', 8: 'l1', 6: 'l1', 12: 'l2', 5: 'l3'}, 'nc2': {}, 'nc3': {12: 'l1', 24: 'l1', 18: 'l1', 36: 'l2', 13: 'l3', 5: 'l2', 4: 'l3', 6: 'l3', 9: 'l3', 16: 'l3'}, 'nc4': {}, 'nc5': {4: 'l2', 9: 'l1', 6: 'l3', 8: 'l1', 13: 'l2', 5: 'l3'}}


In [47]:
l={}
for nc,di in sd.items():
    if len(di) != 0:
        ma=max(di.keys())
        l[nc]=di[ma]
l    

{'nc1': 'l2', 'nc3': 'l2', 'nc5': 'l2'}

In [45]:
ncd={"nc1":2 ,"nc2":0.5 ,"nc3":6 ,"nc4":0.5,"nc5":2.2}
ked={"l1":[2, 0.2, 4.1, 3,4],"l2":[6, 2.2 , 0.9],"l3":[0.7,1,2.3,1.5,2.8]}
sd={}
for i in ncd:
    k={}
    for j in ked:
        l=[]
        for m in ked[j]:
            ss=ncd[i]*m
            if ss > 3:
                l.append(ss)
        if len(l)!=0:
            k[j]=max(l)        
    if len(k)!=0:
        #print(k)
        sd[i]=max(k,key=k.get)
        
print(sd)
# d=swapDict(sd)
# print(d)

{'nc1': 'l2', 'nc3': 'l2', 'nc5': 'l2'}
